In [52]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

True

In [53]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
model_name = "gemini-pro"

llm = GoogleGenerativeAI(model=model_name,
                         google_api_key=GOOGLE_API_KEY,
                         max_output_tokens=2048,
                         max_retries=20,
                         temperature=0)
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [54]:
llm("Hello, how are you today?")

'As an AI chatbot, I don\'t have personal feelings or experiences, so I don\'t have a "today" or the capacity to feel emotions. I am designed to provide information and assist users to the best of my abilities.'

Ingest

In [55]:
from datasets import (
    load_dataset,
)

In [56]:
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE_Philo = os.getenv("ASTRA_DB_KEYSPACE_Philo")

In [57]:
vstore = AstraDBVectorStore(
    embedding=embeddings,
    collection_name="astra_vector_demo",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE_Philo,
)

In [61]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]

docs = []
for entry in philo_dataset:
    # print(entry)
    author = entry["author"]
    quote = entry["quote"]
    tags = entry["tags"]
    metadata = {"author": author, "tags": tags}

    doc = Document(page_content=f"Quote: {quote}, Author:{author}, tags:{str(tags)}", metadata=metadata)
    # print(doc)
    docs.append(doc)

inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")